In [2]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

###### Question 1
- make a function with signature:
- crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)

In [2]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    
    img_copy = img.copy()
    height, width = img.shape[:2]
    
    # M and N are basically number of pixels per patch
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)

    x1, y1 = 0, 0
    
    for y in range(0, height, M):
        for x in range(0, width, N):
            
            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N
        
            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            
            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
        
            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    
    cv.imshow('patched image', img_copy)
    cv.waitKey(0)
    cv.destroyAllWindows() 

In [3]:
img = cv.imread('dog.jfif')
num_horizontal_grid = 3    
num_vertical_grid = 3
line_color = (0, 255, 0)

crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)

###### Question 2
- Display image sequences of smooth transition of two images with different values of α. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.
- for loop

In [6]:
def emptyFunction(self):
    pass

In [7]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

output = cv.addWeighted(img1, 0.5, img2, 0.5, 0)

windowName = "Transition Demo"
cv.namedWindow(windowName)

cv.createTrackbar('Alpha', windowName, 0, 10, emptyFunction)

while(True):
    cv.imshow(windowName, output)
    
    if cv.waitKey(1) == 27:
         break
    
    alpha = cv.getTrackbarPos('Alpha', windowName) / 10
    beta = 1 - alpha
    
    output = cv.addWeighted(img1, alpha, img2, beta, 0)

cv.waitKey(0)
cv.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2581: error: (-27:Null pointer) NULL window: 'Transition Demo' in function 'cvGetTrackbarPos'


###### Question 3
- Rotate image by 45 degrees without cropping the sides of the image.
- Use "lena.jfif" as the input image.
- tips: Use external libraries imutils. / Modify the transformation matrix.

In [17]:
import imutils

img = cv.imread('lena.jfif')

rotate_image = imutils.rotate_bound(img, 45)

window_name='rotated_image'
cv.namedWindow(window_name, cv.WINDOW_NORMAL)

cv.imshow(window_name, rotate_image)
cv.waitKey(0)
cv.destroyAllWindows()

###### Question 4
- Use the images with titles: "flower.jfif" and "native-bee.png".
- I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section.
- But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. 
- The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower.

In [10]:
# Load two images
img1 = cv.imread('native-bee.png')
img2 = cv.imread('flower.jfif')

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols]

# Now create a mask of logo and create its inverse mask also
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 70, 255, cv.THRESH_BINARY) ###change the 2nd value, solve the 'black ink' problem
mask_inv = cv.bitwise_not(mask)

# Now black-out the area of logo in ROI
img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img2_fg = cv.bitwise_and(img2,img2,mask = mask)

# Put logo in ROI and modify the main image
dst = cv.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst

cv.imshow('res',img1)
cv.waitKey(0)
cv.destroyAllWindows()